Тип используемой разметки

- *B-{label}* - начало сущности *{label}*;
- *I-{label}* - продолжение сущности *{label}*;
- *O* - отсутсвие сущности.

Здесь в качестве сущности *{label}* может выступать имя, географическое название или какой-то другой тип собственных имён.

Например, мы хотим извлечь имена и названия организаций. Тогда для текста

    Yan    Goodfellow  works  for  Google Corp Brain

модель должна извлечь следующую последовательность:

    B-PER  I-PER       O      O    B-ORG  I-ORG I-ORG

In [3]:
import torch
import pandas as pd
import torch.utils
import numpy as np
from tqdm import tqdm,trange
import random
from collections import Counter, defaultdict

In [5]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
valid_df = pd.read_csv('data/valid.csv')

In [16]:
train_sent, train_label = [i.split() for i in train_df['sent']],[i.split() for i in train_df['lable']]
test_sent, test_label = [i.split() for i in test_df['sent']],[i.split() for i in test_df['lable']]
valid_sent, valid_label = [i.split() for i in valid_df['sent']],[i.split() for i in valid_df['lable']]

Устанавливаем один и тот же сид для воспроизводимости результатов

In [9]:
def set_global_seed(seed:int)->None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark=False
    torch.backends.cudnn.determnistic = True
set_global_seed(42)

Проинициализируем device (CPU / GPU) на котором будем работать:

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Подготовка словаря лейблов

{**label**}→{**label_idx**}: соответствие между тегом и уникальным индексом (начинается с 0).

In [17]:
def get_label2idx(label_seq):
    label2idx = {}
    label_list = set(label for sentence in label_seq for label in sentence)
    label_list = sorted(label_list, key = lambda x: 'A' if x=='O' else x)
    for i, label in enumerate(label_list):
        label2idx[label] = i
    return label2idx
            

In [18]:
label2idx = get_label2idx(train_label)
label2idx

{'O': 0,
 'B-LOC': 1,
 'B-MISC': 2,
 'B-ORG': 3,
 'B-PER': 4,
 'I-LOC': 5,
 'I-MISC': 6,
 'I-ORG': 7,
 'I-PER': 8}

In [19]:
@staticmethod
def process_labels(labels,label2idx):
    label_ids = list()
    label_ids = [label2idx[label] for label in labels]
    return label_ids

Загрузка модели

In [20]:
from transformers import AutoTokenizer
model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

ModuleNotFoundError: No module named 'transformers'

### Подготовка датасета и загрузчика

Мы также хотим обучать модель батчами, поэтому нам как и прежде понадобятся `Dataset`, `Collator` и `DataLoader`.

Но мы не можем переиспользовать те, что в предыдущих частях задания, так как обработка данных должна производится немного иначе с использованием токенизатора.

Давайте напишем новый кастомный датасет, который на вход (метод `__init__`) будет принимать:
- token_seq - список списков слов / токенов
- label_seq - список списков тегов

и возвращать из метода `__getitem__` два списка:
- список текстовых значений (`List[str]`) из индексов токенов в сэмпле
- список целочисленных значений (`List[int]`) из индексов соответвующих тегов

In [19]:
class TransformeDataset(torch.utils.data.Dataset):
    def __init__(self, token_seq, label_seq):
        self.token_seq = token_seq
        self.label_seq = [self.process_labels(labels,label2idx) for labels in label_seq]
    def __len__(self):
        return len(self.token_seq)
    def __getitem__(self, idx):
        tokens = self.token_seq[idx]
        labels = self.label_seq[idx]
        return tokens,labels
    @staticmethod
    def process_labels(labels,label2idx):
        ids = [label2idx[i] for i in labels]
        return ids        

In [20]:
train_dataset = TransformeDataset(token_seq=train_sent, label_seq=train_label)
valid_dataset = TransformeDataset(token_seq=valid_sent, label_seq=valid_label)
test_dataset = TransformeDataset(token_seq=test_sent, label_seq=test_label)

Реализуем новый `Collator`.

Инициализировать коллатор будет 3 аргументами:
- токенизатор
- параметры токенизатора в виде словаря (затем используем как `**kwargs`)
- id спецтокена для последовательностей тегов (значение -1)

Метод `__call__` на вход принимает батч, а именно список кортежей того, что нам возвращается из датасета. В нашем случае это список кортежей двух int64 тензоров - `List[Tuple[torch.LongTensor, torch.LongTensor]]`.

На выходе мы хотим получить два тензора:
- западденные индексы слов / токенов
- западденные индексы тегов

In [21]:
from transformers import PreTrainedTokenizer
from transformers.tokenization_utils_base import BatchEncoding

class TransformesCollator:
    def __init__(self, tokenizer,tokenizer_kwargs,label_padding_value):
        self.tokenizer = tokenizer
        self.tokenizer_kwargs = tokenizer_kwargs
        self.label_padding_value = label_padding_value
    def __call__(self, batch):
        tokens,labels = zip(*batch)
        tokens = self.tokenizer(list(tokens),**self.tokenizer_kwargs)
        labels =self.encode_labels(tokens,labels,self.label_padding_value)
        tokens.pop('offset_mapping')
        return tokens,labels
    @staticmethod
    def encode_labels(tokens, labels,label_padding_value):
        encoded_labels = []
        for doc_labels, doc_offset in zip(labels, tokens.offset_mapping):
            doc_enc_labels = np.ones(len(doc_offset), dtype=int)*label_padding_value
            arr_offset = np.array(doc_offset)
            
            doc_enc_labels[(arr_offset[: ,0] ==0) & (arr_offset[: ,1]!=0)]=doc_labels
            encoded_labels.append(doc_enc_labels.tolist())
        return torch.LongTensor(encoded_labels)

In [22]:
tokenizer_kwargs = {
    'is_split_into_words': True,
    'return_offsets_mapping':True,
    'padding': True,
    'truncation': True,
    'max_length': 512,
    'return_tensors': 'pt',
}

In [23]:
collator = TransformesCollator(tokenizer =tokenizer, tokenizer_kwargs=tokenizer_kwargs,label_padding_value=-1)

In [24]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=5, shuffle=True,collate_fn=collator)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle=False, shuffle=False, collate_fn=collator,)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False,collate_fn=collator)

В библиотеке **transformers** есть классы для модели BERT, уже настроенные под решение конкретных задач, с соответствующими головами классификации. Для задачи NER будем использовать класс `BertForTokenClassification`.

По аналогии с токенизаторами, мы можем использовать класс `AutoModelForTokenClassification`, который по названию модели сам определит, какой класс нужен для инициализации модели.

In [26]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels = len(label2idx)).to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['cl

In [27]:
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4,eps=1e-8)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

In [28]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=f'logs/Transformers')

In [29]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

Функция подсчета метрики

In [30]:
def compute_metrics(outputs, labels):
    metrics = {}
    mask = (labels!=-1)
    y_true = labels[mask].cpu().numpy()
    y_pred = outputs.argmax(1)
    y_pred = y_pred[mask].cpu().numpy()
    metrics['accuracy'] = accuracy_score(y_true = y_true, y_pred = y_pred)
    for metric_func in [precision_score, recall_score, f1_score]:
        metric_name = metric_func.__name__.split('_')[0]
        for average_type in ['micro','macro','weighted']:
            metrics[metric_name+'_'+average_type] = metric_func( y_true=y_true, y_pred=y_pred,average=average_type,zero_division =0)
    return metrics
    

Функция обучения (получение аутпутов, подсчет лоссов и метрик)

In [31]:
def train_epoch(model,dataloader,optimizer,criterion,writer,device,epoch):
    model.train()
    epoch_loss = []
    batch_metrics_list = defaultdict(list)
    for i, (tokens,labels) in tqdm(enumerate(dataloader),total=len(dataloader),desc='loop over train batches'):
        tokens, labels = tokens.to(device),labels.to(device)
        outputs=None
        loss = None
        optimizer.zero_grad()
        outputs = model(**tokens)['logits'].transpose(1,2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        writer.add_scalar( tag = 'batch loss/train', scalar_value = loss.item(), global_step = epoch*len(dataloader) + i)
        with torch.no_grad():
            model.eval()
            outputs_inference = model(**tokens)['logits'].transpose(1,2)
        batch_metrics = compute_metrics(outputs = outputs_inference, labels=labels)
        for metric_name, metric_value in batch_metrics.items():
            batch_metrics_list[metric_name].append(metric_value)
            writer.add_scalar(tag = f'batch {metric_name}/train', scalar_value = metric_value, global_step=epoch*len(dataloader) + i)
    avg_loss = np.mean(epoch_loss)
    print(f'Train loss: {avg_loss}\n')
    for metric_name, metric_value_list in batch_metrics_list.items():
        metric_value = np.mean(metric_value_list)
        print(f'Train {metric_name}: {metric_value}\n')

Функция проверки точности на тестовом датасете

In [32]:
def evaluate_epoch(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    writer: SummaryWriter,
    device: torch.device,
    epoch: int,
) :
    model.eval()
    epoch_loss = []
    batch_metrics_list = defaultdict(list)
    with torch.no_grad():

        for i, (tokens, labels) in tqdm(enumerate(dataloader),total=len(dataloader),desc="loop over test batches",):

            tokens, labels = tokens.to(device), labels.to(device)
            outputs = model(**tokens)["logits"].transpose(1, 2)
            loss = criterion(outputs, labels)
            epoch_loss.append(loss.item())
            writer.add_scalar(tag="batch loss / test",scalar_value=loss.item(),global_step=epoch * len(dataloader) + i,)
            batch_metrics = compute_metrics(
                outputs=outputs,
                labels=labels,
            )

            for metric_name, metric_value in batch_metrics.items():
                batch_metrics_list[metric_name].append(metric_value)
                writer.add_scalar(
                    tag=f"batch {metric_name} / test",
                    scalar_value=metric_value,
                    global_step=epoch * len(dataloader) + i,
                )
        avg_loss = np.mean(epoch_loss)
        print(f"Test loss:  {avg_loss}\n")
        writer.add_scalar(tag="loss / test",scalar_value=avg_loss,global_step=epoch,)
        for metric_name, metric_value_list in batch_metrics_list.items():
            metric_value = np.mean(metric_value_list)
            print(f"Test {metric_name}: {metric_value}\n")
            writer.add_scalar(
                tag=f"{metric_name} / test",
                scalar_value=np.mean(metric_value),
                global_step=epoch,
            )

In [33]:
def train(n_epochs, model, train_dataloader, valid_dataloader,optimizer,criterion,writer,device):
    for epoch in range(n_epochs):
        train_epoch(model=model ,dataloader = train_dataloader, optimizer=optimizer, criterion=criterion, writer=writer, device=device, epoch=epoch)
        evaluate_epoch(model=model ,dataloader = valid_dataloader, optimizer=optimizer, criterion=criterion, writer=writer, device=device, epoch=epoch)

In [34]:
train(n_epochs=1,model=model,train_dataloader=train_dataloader,optimizer=optimizer,criterion=criterion,writer=writer,device=device)

loop over train batches: 100%|██████████| 3/3 [00:11<00:00,  3.95s/it]

Train loss: 0.4289773851633072

Train accuracy: 0.9029791761665907

Train precision_micro: 0.9029791761665907

Train precision_macro: 0.6177871148459383

Train precision_weighted: 0.8464564527741482

Train recall_micro: 0.9029791761665907

Train recall_macro: 0.5333333333333333

Train recall_weighted: 0.9029791761665907

Train f1_micro: 0.9029791761665907

Train f1_macro: 0.5298592271055647

Train f1_weighted: 0.8621313709902693



In [35]:
evaluate_epoch(model=model,
    dataloader=test_dataloader,
    criterion=criterion,
    writer=writer,
    device=device,
    epoch=1)

loop over test batches: 100%|██████████| 5/5 [00:00<00:00,  8.43it/s]

Test loss:  0.19298928380012512

Test accuracy: 0.8873737373737374

Test precision_micro: 0.8873737373737374

Test precision_macro: 0.4436868686868687

Test precision_weighted: 0.7875886389143965

Test recall_micro: 0.8873737373737374

Test recall_macro: 0.5

Test recall_weighted: 0.8873737373737374

Test f1_micro: 0.8873737373737374

Test f1_macro: 0.470140056022409

Test f1_weighted: 0.8344673627026568

